In [ ]:
# !pip install opentelemetry-api
# !pip install opentelemetry-sdk

In [ ]:
from pathlib import Path
import sys

# Add base dir to path to be able to access test folder.
base_dir = Path().cwd().parent.parent.resolve()
if str(base_dir) not in sys.path:
    print(f"Adding {base_dir} to sys.path")
    sys.path.append(str(base_dir))

In [ ]:
from trulens.experimental.otel_tracing.core.instrument import instrument


class TestApp:
    @instrument()
    def respond_to_query(self, query: str) -> str:
        return f"answer: {self.nested(query)}"

    @instrument(attributes={"nested_attr1": "value1"})
    def nested(self, query: str) -> str:
        return f"nested: {self.nested2(query)}"

    @instrument(
        attributes=lambda ret, *args, **kwargs: {
            "nested2_ret": ret,
            "nested2_args[0]": args[0],
        }
    )
    def nested2(self, query: str) -> str:
        return f"nested2: {query}"

In [ ]:
import os

import dotenv
from trulens.connectors.snowflake import SnowflakeConnector
from trulens.core.session import TruSession
from trulens.experimental.otel_tracing.core.init import init

dotenv.load_dotenv()

connection_params = {
    "account": os.environ["SNOWFLAKE_ACCOUNT"],
    "user": os.environ["SNOWFLAKE_USER"],
    "password": os.environ["SNOWFLAKE_USER_PASSWORD"],
    "database": os.environ["SNOWFLAKE_DATABASE"],
    "schema": os.environ["SNOWFLAKE_SCHEMA"],
    "warehouse": os.environ["SNOWFLAKE_WAREHOUSE"],
    "role": os.environ["SNOWFLAKE_ROLE"],
}

connector = SnowflakeConnector(
    **connection_params, database_redact_keys=True, database_args=None
)
session = TruSession(connector=connector)
session.reset_database()
init(session, debug=True)

In [ ]:
from trulens.apps.custom import TruCustomApp

test_app = TestApp()
custom_app = TruCustomApp(test_app)

with custom_app as recording:
    test_app.respond_to_query("test")